#### Лабораторная работа 3. Начальная фаза симплекс-метода
Реализовать начальную фазу симплекс-метода. Определить совместна ли задача и, в случае положительного ответа, найти
какой-нибудь базисный допустимый план $(x, B)$.

$----------------------------------------------$

In [831]:
import numpy as np

<u>Шаг 0.</u> Определение исходных данных задачи (1)

In [832]:
c = np.array([1, 0, 0])
b = np.array([0, 0])
A = np.array([[1, 1, 1], [2, 2, 2]])

n = c.size
m = b.size

<u>Шаг 1.</u> Необходимо преобразовать задачу таким образом, чтобы вектор правых частей $b$ был неотрицательным. Для этого умножим на $−1$ все
ограничения задачи, правая часть которых отрицательна. А именно, для каждого индекса $i \in \{1, 2, . . . , m\}$ выполним следующую операцию: если $b_{i} < 0$, то умножим на $−1$ компоненту $b_{i}$ и $i$-ю строку матрицы $A$

In [833]:
for i in range(0, m):
    if b[i] < 0:
        b[i] = b[i] * -1
        A[i, :] = A[i, :] * -1

print("A =", A)

A = [[1 1 1]
 [2 2 2]]


<u>Шаг 2.</u> Составим вспомогательную задачу (2) линейного программирования

${\tilde{c}}^{T} \tilde{x} \rightarrow max$

$\tilde{A} \tilde{x} = b$                           

$\tilde{x} \ge 0$

где вектор коэффициентов при переменных в целевой функции имеет вид

$\tilde{c}^{T}=(\underbrace{0,0,...,0}_{n},\underbrace{-1,-1,...,-1}_{m}) \in \R^{n+m}$

вектор переменных — $\tilde{x} = (x_{1},x_{2},...,x_{n},x_{n+1},x_{n+2},...,x_{n+m})^{T} \in \R^{n+m}$ (переменные $x_{n+1},x_{n+2},...,x_{n+m}$ называются искусственными), матрица $\tilde{A}$ получается из матрицы $A$ присоединением к ней справа единичной матрицы порядка $m$.

Тогда ${\tilde{c}}^{T}$ имеет вид:


In [834]:
tilde_t_c = []
for i in range(0, n):
    tilde_t_c.append(0)
    
for i in range(0, m):
    tilde_t_c.append(-1)
    
tilde_t_c = np.array(tilde_t_c)

print("tilde_T_c =", tilde_t_c)

tilde_T_c = [ 0  0  0 -1 -1]


А матрица $\tilde{A}$ имеет вид:

In [835]:
tilde_A = np.hstack((A, np.eye(m)))
print("tilde_A =", tilde_A)

tilde_A = [[1. 1. 1. 1. 0.]
 [2. 2. 2. 0. 1.]]


<u>Шаг 3.</u> Построим начальный базисный допустимый план $(\tilde{x}, B)$ задачи (2)

$\tilde{x}=(0, 0, ..., 0, b_{1}, b_{2}, ..., b_{m}) \in \R^{n+m}$

$B = \{j_{1}=n+1, j_{2}=n+2, ..., j_{m}=n+m\}$

Тогда $\tilde{x}$ будет выглядеть следующим образом:

In [836]:
tilde_x = []
for i in range(0, n):
    tilde_x.append(0)
    
tilde_x = np.hstack((np.array(tilde_x), b))

tilde_x

array([0, 0, 0, 0, 0])

А $B$:

In [837]:
B = []
for i in range(1, m + 1):
    B.append(n + i)

B = np.array(B)
    
print("B =", B)

B = [4 5]


<u>Шаг 4.</u> Решим вспомогательную задачу (2) основной фазой симплекс-метода и получим оптимальный план

$\tilde{x} = (\tilde{x}_{1}, \tilde{x}_{2}, ..., \tilde{x}_{n}, \tilde{x}_{n + 1}, \tilde{x}_{n + 2}, ..., \tilde{x}_{n + m})^{T}$

и соответствующее ему множество базисных индексов $B$.

In [838]:
a = tilde_A
c_t = tilde_t_c
x_t = tilde_x

a_str_size = n + m
a_row_size = m

while(True):
    # Шаг 1. ===============================
    a_b = np.zeros((a_row_size, B.size))

    row_index = 0
    for i in B:
        a_b[:, row_index] = [x[i - 1] for x in a]
        row_index += 1
        
    a_b_invert = np.linalg.inv(a_b)

    # Шаг 2. ===============================
    c_b_elems = []

    for i in B:
        c_b_elems.append(c_t[i - 1])

    c_b = np.array(c_b_elems)

    # Шаг 3. ===============================
    u_t = l = np.dot(c_b, a_b_invert)

    # Шаг 4. ===============================
    delta_t = np.dot(u_t, a) - c_t

    # Шаг 5. ===============================
    if delta_t[delta_t < 0].size == 0:
        print("x =", x_t, "is optimal")
        tilde_x = x_t
        print("B =", B)
        break

    # Шаг 6. ===============================
    j = np.argmax(delta_t < 0) + 1

    # Шаг 7. ===============================
    z = np.dot(a_b_invert, [x[j - 1] for x in a])

    # Шаг 8. ===============================
    teta_t = []
    counter = 0

    for i in z:
        if i <= 0:
            teta_t.append(np.Inf)
        else:
            teta_t.append(x_t[b[counter] - 1])
        counter += 1
        
    teta_t = np.array(teta_t)

    # Шаг 9. ===============================
    min_elem = np.amin(teta_t)

    # Шаг 10. ==============================
    if min_elem == np.Inf:
        print("Value is not limited from above")
        break
        
    # Шаг 11. ==============================
    counter = 0
    for i in teta_t:
        if i == min_elem:
            break
        counter += 1
        
    # Шаг 12. ==============================
    B[counter] = j

    # Шаг 13. ==============================
    counter_2 = 0
    for i in B:
        if counter == counter_2:
            x_t[j - 1] = min_elem
        else:
            x_t[i - 1] = x_t[i - 1] - min_elem * z[counter_2]
        
        counter_2 += 1

    for i in range(0, x_t.size):
        if not np.isin(b, i + 1).any():
            x_t[i] = 0

x = [0 0 0 0 0] is optimal
B = [1 5]


Шаг 5. Проверим условия совместности: если $\tilde{x}_{n+1} = \tilde{x}_{n+2} = ... = \tilde{x}_{n+m} = 0$ то задача (1) совместна; в противном случае, задача (1) не совместна и метод завершает свою работу.

In [839]:
joint = True
for i in range(n, m + n):
    if tilde_x[i] != 0:
        joint = False
        break
    
if joint:
    print("The problem (1) is joint")
else:
    print("The problem (1) is not joint")



The problem (1) is joint


Шаг 6. Формируем допустимый план $x = (\tilde{x}_{1}, \tilde{x}_{2}, ..., \tilde{x}_{n})$ задачи (1). Для него необходимо подобрать множество базисных индексов. С этой целью скорректируем множество B следующим образом.

In [840]:
x = []

for i in range(0, n):
    x.append(tilde_x[i])

x = np.array(x)

print("x =", x)

x = [0 0 0]


<u>Шаг 7 - 11.</u> Корректирующий алгоритм (цикл):


$\boldsymbol{\cdot}$ Если $B \subseteq \{1, 2, . . . , n\}$, то алгоритм завершает свою работу и возвращает базисный допустимый план $(x, B)$

$\boldsymbol{\cdot}$ Выберем в наборе $B$ максимальный индекс искусственной переменной  $j_{k} = n + i$

$\boldsymbol{\cdot}$ Для каждого индекса $j \in \{1, 2, . . . , n\} \backslash B$ вычислим вектор $ℓ(j) = \tilde{A}^{-1}_{B} \tilde{A}_{j}$ , где $\tilde{A}_{j}$ — это $j$-ый столбец матрицы $\tilde{A}$.

$\boldsymbol{\cdot}$ Если найдется индекс $j \in \{1, 2, . . . , n\} \backslash B$ такой, что $(ℓ(j))_{k} \ne 0$, то заменим в наборе $B$ значение $j_{k}$, равное $n + i$, на $j$.

$\boldsymbol{\cdot}$ Если для любого индекса $j \in \{1, 2, . . . , n\} \backslash B$ выполняется $(ℓ(j))_{k} = 0$, то $i$-е основное ограничение задачи (1) линейно выражается через остальные и его необходимо удалить. В этом случае удалим $i$-ую строку из матрицы $A$ и $i$-ую компоненту из вектора $b$. Удалим из $B$ индекс $j_{k} = n + i$. Кроме этого, удалим $i$-ую строку из матрицы $\tilde{A}$

In [841]:
while(True):
    #1 ==============
    subset = [i for i in range(1, n + 1)]
    exit_flag = True
    for B_elem in B:
        if B_elem not in subset:
            exit_flag = False
    if exit_flag:
        break
    
    #2 ==============
    k = np.argmax(B)
    j_k = B[k]
    i = j_k - n
    
    #3 ==============
    tilde_A_B = np.empty([m, B.size])

    counter = 0
    for b_index in B:
        tilde_A_B[:, [counter]] = tilde_A[:, [b_index - 1]]
        counter += 1
    
    for elem in B:
        if elem in subset:
            subset.remove(elem)
    
    l = []
    tilde_A_inv = np.linalg.inv(tilde_A_B)
    for index in subset:
        l_j = np.dot(tilde_A_inv, tilde_A[0:n, [index - 1]])
        l.append(l_j)
    
    #4 ==============
    e_zero = True
    counter = 0
    
    l = np.array(l)
    for vec in l:
        if vec[k][0] != 0:
            B[k] = subset[counter]
            ne_zero = False
        counter += 1
        
    #5 ==============
    if e_zero:
        A = np.delete(A, i - 1, 0)
        b = np.delete(b, i - 1)
        tilde_A = np.delete(tilde_A, i - 1)
        B = np.delete(B, k)
        
print("x =", x)
print("B =", B)
print("A =", A)
print("b =", b)


x = [0 0 0]
B = [1]
A = [[1 1 1]]
b = [0]
